<a href="https://colab.research.google.com/github/rakeshgovindsuresh/Python-Practice/blob/main/Feature%20Engg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/cleaned_dataset.csv'  # Replace with the actual path to your dataset
dataset = pd.read_csv(file_path)

# 1. Batsman Statistics

# Total Runs, Balls Faced, Number of 4s and 6s
batsman_stats = dataset.groupby('batter').agg(
    total_runs=('batsman_runs', 'sum'),
    balls_faced=('ball', 'count'),
    fours=('batsman_runs', lambda x: (x == 4).sum()),
    sixes=('batsman_runs', lambda x: (x == 6).sum())
).reset_index()

# Calculate Strike Rate for Batsman
batsman_stats['strike_rate'] = (batsman_stats['total_runs'] / batsman_stats['balls_faced']) * 100

# 2. Bowler Statistics

# Total Wickets, Runs Conceded, and Balls Bowled (to calculate overs)
bowler_stats = dataset.groupby('bowler').agg(
    total_wickets=('is_wicket', 'sum'),
    runs_conceded=('total_runs', 'sum'),
    balls_bowled=('ball', 'count')
).reset_index()

# Calculate Overs Bowled and Economy Rate
bowler_stats['overs_bowled'] = bowler_stats['balls_bowled'] / 6
bowler_stats['economy_rate'] = bowler_stats['runs_conceded'] / bowler_stats['overs_bowled']

# 3. Batsman vs. Bowler Comparison

# Average Runs per Ball (Batsman vs. Bowler)
batsman_bowler_avg_runs = dataset.groupby(['batter', 'bowler'])['batsman_runs'].mean().reset_index()
batsman_bowler_avg_runs.columns = ['batter', 'bowler', 'avg_runs_per_ball_batsman_vs_bowler']

# Strike Rate (Batsman vs. Bowler)
batsman_bowler_stats = dataset.groupby(['batter', 'bowler']).agg(
    total_runs=('batsman_runs', 'sum'),
    total_balls=('ball', 'count')
).reset_index()
batsman_bowler_stats['strike_rate_batsman_vs_bowler'] = (batsman_bowler_stats['total_runs'] / batsman_bowler_stats['total_balls']) * 100

# Dismissal Rate (Batsman vs. Bowler)
dismissals = dataset[dataset['is_wicket'] == 1].groupby(['batter', 'bowler']).size().reset_index(name='dismissals')
dismissal_rate = dismissals.merge(batsman_bowler_stats[['batter', 'bowler', 'total_balls']], on=['batter', 'bowler'])
dismissal_rate['dismissal_rate_batsman_vs_bowler'] = dismissal_rate['total_balls'] / dismissal_rate['dismissals']

# 4. Percentage of Runs Distribution per Batsman vs. Bowler

# Calculate the percentage of balls that resulted in 6s, 4s, 3s, 2s, 1s, and 0 runs
runs_distribution = dataset.groupby(['batter', 'bowler', 'batsman_runs']).size().unstack(fill_value=0).reset_index()

# Calculate percentages for each type of run
numeric_columns = runs_distribution.select_dtypes(include='number').columns  # Only sum numeric columns
total_balls_faced = runs_distribution[numeric_columns].sum(axis=1)
runs_distribution['percent_6s'] = (runs_distribution[6] / total_balls_faced) * 100
runs_distribution['percent_4s'] = (runs_distribution[4] / total_balls_faced) * 100
runs_distribution['percent_3s'] = (runs_distribution[3] / total_balls_faced) * 100
runs_distribution['percent_2s'] = (runs_distribution[2] / total_balls_faced) * 100
runs_distribution['percent_1s'] = (runs_distribution[1] / total_balls_faced) * 100
runs_distribution['percent_0s'] = (runs_distribution[0] / total_balls_faced) * 100

# Calculate the percentage of dismissals
dismissal_percentage = dismissal_rate.copy()
dismissal_percentage['percent_dismissals'] = (dismissal_percentage['dismissals'] / dismissal_percentage['total_balls']) * 100

# Merge percentage distributions with the main batsman-bowler comparison DataFrame
batsman_bowler_comparison = batsman_bowler_avg_runs.merge(
    batsman_bowler_stats[['batter', 'bowler', 'strike_rate_batsman_vs_bowler']],
    on=['batter', 'bowler']
).merge(
    dismissal_rate[['batter', 'bowler', 'dismissal_rate_batsman_vs_bowler']],
    on=['batter', 'bowler'], how='left'
).merge(
    runs_distribution[['batter', 'bowler', 'percent_6s', 'percent_4s', 'percent_3s', 'percent_2s', 'percent_1s', 'percent_0s']],
    on=['batter', 'bowler'], how='left'
).merge(
    dismissal_percentage[['batter', 'bowler', 'percent_dismissals']],
    on=['batter', 'bowler'], how='left'
)

# 5. Contextual Features

# Over Context (Powerplay, Middle Overs, Death Overs)
def categorize_over(over):
    if over <= 6:
        return 'Powerplay'
    elif over <= 15:
        return 'Middle Overs'
    else:
        return 'Death Overs'

dataset['over_context'] = dataset['over'].apply(categorize_over)

# Required Run Rate
dataset['required_run_rate'] = (dataset['target_runs'] - dataset.groupby('match_id')['total_runs'].cumsum()) / (dataset['target_overs'] * 6 - dataset.groupby('match_id')['ball'].cumsum())

# Current Run Rate
dataset['current_run_rate'] = dataset.groupby('match_id')['total_runs'].cumsum() / (dataset['over'] + dataset['ball'] / 6)

# Wickets in Hand
dataset['wickets_in_hand'] = 10 - dataset.groupby('match_id')['is_wicket'].cumsum()

# 6. Batsman’s Recent Form Features

# Batsman’s Recent Average Runs
def calculate_recent_average(df, window=20):
    df['recent_avg_runs'] = df.groupby('batter')['batsman_runs'].rolling(window).mean().reset_index(level=0, drop=True)
    return df

dataset = calculate_recent_average(dataset)

# Batsman’s Recent Boundary Rate
def calculate_recent_boundaries(df, window=20):
    df['is_boundary'] = df['batsman_runs'].isin([4, 6])
    df['recent_boundary_rate'] = df.groupby('batter')['is_boundary'].rolling(window).mean().reset_index(level=0, drop=True)
    return df

dataset = calculate_recent_boundaries(dataset)

# 7. Bowler’s Recent Performance Features

# Bowler’s Recent Economy Rate
def calculate_recent_economy(df, window=12):  # Assuming an over contains 6 balls
    df['recent_runs_conceded'] = df.groupby('bowler')['total_runs'].rolling(window).sum().reset_index(level=0, drop=True)
    df['recent_economy_rate'] = df['recent_runs_conceded'] / (window / 6)
    return df

dataset = calculate_recent_economy(dataset)

# Bowler’s Recent Wicket-Taking Ability
def calculate_recent_wickets(df, window=12):
    df['recent_wickets'] = df.groupby('bowler')['is_wicket'].rolling(window).sum().reset_index(level=0, drop=True)
    return df

dataset = calculate_recent_wickets(dataset)

# 8. Partnership Information

# Calculate Partnership Runs and Partnership Balls Faced
dataset['partnership_runs'] = dataset.groupby(['match_id', 'inning', 'batting_team'])['total_runs'].cumsum() - dataset['batsman_runs']
dataset['partnership_balls_faced'] = dataset.groupby(['match_id', 'inning', 'batting_team']).cumcount() + 1

# 9. Match Pressure Metrics

# Pressure Factor = Required Run Rate / (Wickets in Hand + 1)
dataset['pressure_factor'] = dataset['required_run_rate'] / (dataset['wickets_in_hand'] + 1)

# Merge Everything into One DataFrame

# Merge batsman stats
final_dataset = dataset.merge(batsman_stats, left_on='batter', right_on='batter', how='left')

# Merge bowler stats
final_dataset = final_dataset.merge(bowler_stats, left_on='bowler', right_on='bowler', how='left')

# Merge batsman vs. bowler comparison
final_dataset = final_dataset.merge(batsman_bowler_comparison, on=['batter', 'bowler'], how='left')

# Save the final dataset to a CSV file
final_file_path = '/content/newFeatureEngdataset.csv'
final_dataset.to_csv(final_file_path, index=False)

print(f"The final combined dataset has been saved as {final_file_path}")



<ipython-input-2-98143e734e55>:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(file_path)


The final combined dataset has been saved as /content/newFeatureEngdataset.csv


In [ ]:
import pandas as pd

# Load the datasets
df_csv = pd.read_csv('newFeatureEngdataset.csv', dtype=str, low_memory=False)
df_excel = pd.read_excel('updated_stadium_stats_with_wankhede_stadium (1).xlsx')

# Strip any whitespace from the 'venue' column and convert to lowercase for consistent merging
df_csv['venue'] = df_csv['venue'].str.strip().str.lower()
df_excel['venue'] = df_excel['venue'].str.strip().str.lower()

# Prefix all columns from df_excel except 'venue' with 'venue_'
df_excel = df_excel.rename(columns=lambda x: f"venue_{x}" if x != 'venue' else x)

# Check for unique values in the 'venue' column to identify any mismatches
print("Unique venues in CSV file:", df_csv['venue'].unique())
print("Unique venues in Excel file:", df_excel['venue'].unique())

# Merge the datasets (left join to preserve all rows from df_csv)
merged_df = pd.merge(df_csv, df_excel, on='venue', how='left')

# Save the merged DataFrame
merged_df.to_excel('newmergedFeaturePlusVenueStats.xlsx', index=False)
merged_df.to_csv('newmergedFeaturePlusVenueStats.csv', index=False)

# Print the shape of the merged dataframe to check if rows are preserved
print("Shape of the merged DataFrame:", merged_df.shape)


Unique venues in CSV file: ['m chinnaswamy stadium' 'punjab cricket association stadium, mohali'
 'feroz shah kotla' 'wankhede stadium' 'eden gardens'
 'sawai mansingh stadium' 'rajiv gandhi international stadium, uppal'
 'ma chidambaram stadium, chepauk' 'dr dy patil sports academy' 'newlands'
 "st george's park" 'kingsmead' 'supersport park' 'buffalo park'
 'new wanderers stadium' 'de beers diamond oval' 'outsurance oval'
 'brabourne stadium' 'sardar patel stadium, motera' 'barabati stadium'
 'brabourne stadium, mumbai'
 'vidarbha cricket association stadium, jamtha'
 'himachal pradesh cricket association stadium' 'nehru stadium'
 'holkar cricket stadium'
 'dr. y.s. rajasekhara reddy aca-vdca cricket stadium'
 'subrata roy sahara stadium' 'maharashtra cricket association stadium'
 'shaheed veer narayan singh international stadium'
 'jsca international stadium complex' 'sheikh zayed stadium'
 'sharjah cricket stadium' 'dubai international cricket stadium'
 'punjab cricket association 

In [ ]:
import pandas as pd

# Load the datasets
df_csv = pd.read_csv('/content/newmergedFeaturePlusVenueStats.csv', dtype=str, low_memory=False)


columns_to_drop = ['match_id','team1','team2','venue_City','venue_Country','venue_First Match','venue_Last T20 Match','venue_Also or previously Known as','season','super_over','non_striker','venue_Best Bowling','venue_Highest Individual Innings']

df_csv = df_csv.drop(columns=columns_to_drop)

# Check the remaining columns
print(df_csv.columns)

Index(['venue', 'result_margin', 'target_runs', 'target_overs', 'inning',
       'batting_team', 'bowling_team', 'over', 'ball', 'batter', 'bowler',
       'batsman_runs', 'extra_runs', 'total_runs_x', 'extras_type',
       'is_wicket', 'over_context', 'required_run_rate', 'current_run_rate',
       'wickets_in_hand', 'recent_avg_runs', 'is_boundary',
       'recent_boundary_rate', 'recent_runs_conceded', 'recent_economy_rate',
       'recent_wickets', 'partnership_runs', 'partnership_balls_faced',
       'pressure_factor', 'total_runs_y', 'balls_faced', 'fours', 'sixes',
       'strike_rate', 'total_wickets', 'runs_conceded', 'balls_bowled',
       'overs_bowled', 'economy_rate', 'avg_runs_per_ball_batsman_vs_bowler',
       'strike_rate_batsman_vs_bowler', 'dismissal_rate_batsman_vs_bowler',
       'percent_6s', 'percent_4s', 'percent_3s', 'percent_2s', 'percent_1s',
       'percent_0s', 'percent_dismissals', 'venue_Matches Played',
       'venue_Matches Won Batting First', 'venue_Ma

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder



# List of columns you want to convert to categorical and apply label encoding
columns_to_convert = ['venue', 'batter', 'bowler', 'extras_type', 'over_context', 'is_boundary']

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each column
for column in columns_to_convert:
    df_csv[column] = label_encoder.fit_transform(df_csv[column])

# Verify the changes
print(df_csv.dtypes)
output_csv_path = '/content/newfiletocheckconv.csv'  # Replace with your desired output file path
df_csv.to_csv(output_csv_path, index=False)

venue                                 int64
result_margin                        object
target_runs                          object
target_overs                         object
inning                               object
                                      ...  
venue_Lowest Team Innings            object
venue_Highest Run Chase Achieved     object
venue_Average Runs per Wicket        object
venue_Average Runs per Over          object
venue_Average Score Batting First    object
Length: 61, dtype: object


In [ ]:
import pandas as pd
file_path = "/content/newfiletocheckconv.csv"

df_csv = pd.read_csv(file_path)
columns_to_drop = ['batting_team','bowling_team']
df_csv = df_csv.drop(columns=columns_to_drop)
columns_to_extract = ['venue_Highest Team Innings', 'venue_Lowest Team Innings',
                      'venue_Highest Run Chase Achieved']
for column in columns_to_extract:
    # Extract the numeric part, handle NaN, and convert to integers
    df_csv[column] = df_csv[column].str.extract(r'(\d+)', expand=False)
    df_csv[column] = df_csv[column].fillna(0).astype(int)


# Verify the changes
print(df_csv[columns_to_extract].head())
df_csv.fillna(0, inplace=True)
output_csv_path = '/content/newfullyFeaturedfinalFile.csv'
df_csv.to_csv(output_csv_path, index=False)

<ipython-input-23-9ea4f2a77cb4>:4: DtypeWarning: Columns (50,51,52,53,54,55,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv(file_path)


   venue_Highest Team Innings  venue_Lowest Team Innings  \
0                         287                         82   
1                         287                         82   
2                         287                         82   
3                         287                         82   
4                         287                         82   

   venue_Highest Run Chase Achieved  
0                               186  
1                               186  
2                               186  
3                               186  
4                               186  


In [ ]:
import pandas as pd
import re

# Load your DataFrame from a CSV file
df_csv = pd.read_csv('/content/newfullyFeaturedfinalFile.csv')

# Define a function to extract the percentage values
def extract_percentage(value):
    if isinstance(value, str):
        # Use regex to extract the percentage value inside parentheses
        match = re.search(r'\((\d+\.\d+)%\)', value)
        if match:
            return float(match.group(1))
    return value

# Specify the columns that need to be updated with percentage extraction
columns_to_update = ['venue_Matches Won Batting First', 'venue_Matches Won Batting Second',
                     'venue_Matches Won Winning Toss', 'venue_Matches Won Losing Toss',
                     'venue_Matches with No Result']

# Extract percentage values for the relevant columns
for col in columns_to_update:
    if col in df_csv.columns:  # Ensure the column exists in the dataframe
        df_csv[col] = df_csv[col].apply(extract_percentage)

# Update column headers by appending '(in %)'
df_csv = df_csv.rename(columns={col: col + ' (in %)' for col in columns_to_update})

# Save the updated DataFrame back to a CSV file
df_csv.to_csv('/content/newfullyFeaturedfinalFileUpdated.csv', index=False)

# Display the first few rows of the updated DataFrame as a check
print(df_csv.head())


<ipython-input-24-01905826656a>:5: DtypeWarning: Columns (48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_csv = pd.read_csv('/content/newfullyFeaturedfinalFile.csv')


   venue  result_margin  target_runs  target_overs  inning  over  ball  \
0     23          140.0        223.0          20.0       1     0     1   
1     23          140.0        223.0          20.0       1     0     2   
2     23          140.0        223.0          20.0       1     0     3   
3     23          140.0        223.0          20.0       1     0     4   
4     23          140.0        223.0          20.0       1     0     5   

   batter  bowler  batsman_runs  ...  venue_Matches Won Batting Second (in %)  \
0     537     335             0  ...                                    52.63   
1     102     335             0  ...                                    52.63   
2     102     335             0  ...                                    52.63   
3     102     335             0  ...                                    52.63   
4     102     335             0  ...                                    52.63   

   venue_Matches Won Winning Toss (in %)  \
0                       